In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml
import json
import re
import os
import analysis
from matplotlib.dates import DateFormatter
from datetime import datetime

import openpyxl
from openpyxl.utils import get_column_letter

with open("protrait_data_config.yaml", 'r') as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)

# analysis.metric_list(cfg)

# analysis.preview(
#     cfg,
#     metrics=[
# "host_cpu_usage_system",
# "host_cpu_usage_user",
# "host_memory_mem_bandwidth_local_numa_2",
# "host_memory_mem_bandwidth_total_numa_2",
# "host_memory_usage_not_contain_cache_and_buffer_not_contain_cache_and_buffer",
# "host_memory_usage_contain_cache_and_buffer_contain_cache_and_buffer",
# "vm_hypervisor_emulator_syscall_duration_recvmsg",
# "vm_hypervisor_emulator_syscall_duration_sendmsg",
# "vm_cpu_time_sys",
# "vm_cpu_time_user",
# "vm_cpu_vcpu_time_per_seconds_vcpu",
# "vm_cpu_context_switch_context_switch",
# "vm_cpu_branch_ips",
# "vm_cpu_cycle_per_instruction_cpi",
# "vm_cpu_instructions_per_second_ins",
# "vm_cpu_branch_miss_per_branch_instructions",
# "vm_cache_misses_per_thousand_instructions",
# "vm_cache_last-level_cache_capacity_vm_occupancy_numa_2",
# "vm_mem_usage",
# "vm_mem_bandwidth_local_local_numa_2",
# "vm_mem_bandwidth_local_total_numa_2",
# "vm_network_io_bytes_receive",
# "vm_network_io_bytes_transmit",
# "vm_network_io_packets_receive",
# "vm_network_io_packets_transmit",
# "vm_block_io_bytes_io_write",
# "vm_block_io_requests_write",
# "vm_block_io_time_write",
# "vm_block_io_flush_request_and_time_request_vda",
# "vm_block_io_flush_request_and_time_time_vda",
# "app_redis_qos_qps_of_redis_get",
# "app_redis_qos_qps_of_redis_set",
# "app_redis_qos_qps_of_redis_total",
# "app_redis_qos_p99_latency_set",
# "app_redis_qos_p99_latency_get",
# "app",
# "workload",
# "count",
# "mem_stress",
# "cpu_stress",
# "cpu_load_stress"
#     ]
# )


analysis.protrait_metric(cfg)

开始生成场景mem_stress指标，目标文件data/20231018/mem_stress.xlsx
指标计算完成


In [1]:
analysis.all_metrics_list(cfg)

NameError: name 'analysis' is not defined

In [10]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import textwrap
import csv
import os
import openpyxl
import sys
from openpyxl.utils import get_column_letter
from matplotlib import ticker
from scipy.stats import entropy
from sklearn import tree
from sklearn import svm
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import accuracy_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

def split_rects(obj_count,fig_size_width=24,fig_size_higth = 16,title_space_rate=0.03,bottom_space_rate=0.05,left_space_rate=0.05):
    sq = int(math.sqrt(obj_count))
    higth_count = sq if sq * sq >= obj_count else sq + 1
    width_count = sq if (sq + 1) * sq > obj_count else sq + 1

    title_space = fig_size_higth * title_space_rate
    bottom_space = fig_size_higth * bottom_space_rate
    left_space = fig_size_width * left_space_rate
    higth = ( fig_size_higth - title_space - bottom_space) / higth_count
    width = ( fig_size_width - left_space ) / width_count
    
    higth_spacing = 0.25 * higth
    width_spacing = 0.2 * width
    
    rects_local = []
    for j in range(1,higth_count+1):
        for i in range(1,width_count+1):
            x = ( ( i - 1 ) % width_count ) * width
            y = fig_size_higth - (( ( j - 1 ) % higth_count ) + 1 ) * higth
            rects_local.append(
                ( (x + left_space) / fig_size_width,
                  (y - title_space) / fig_size_higth, 
                 (width -  width_spacing) / fig_size_width,
                 (higth - higth_spacing)  / fig_size_higth
            ))
    return rects_local,width_count,higth_count
    
def compare(data,select_columns=[],label_columns=[],fig_size_width=24,fig_size_higth = 16,fontsize_set = 18,subtitle_width = 25,suby_label_width=25,subhist_buckets = 20,title_space_rate=0.03,bottom_space_rate=0.05,left_space_rate=0.05,sub_dir='test',app=""):
    
  
    # 预处理
    x_column_index = 'stress'
    data = data.sort_values(by=x_column_index)
    data = data.reset_index(drop=True)

    if len(select_columns) == 0:
        metric_count =  len(data.columns) - 1
    else:
        metric_count = len(select_columns)

    rects_local,width_count,higth_count = split_rects(obj_count=metric_count,
                              fig_size_width=fig_size_width,
                              fig_size_higth = fig_size_higth,
                              title_space_rate=title_space_rate,
                              bottom_space_rate=bottom_space_rate,
                              left_space_rate=left_space_rate)

    if len(select_columns) == 0:
        columns = data.columns[1:]
    else:
        columns = select_columns
    
    fig = plt.figure(figsize=(fig_size_width,fig_size_higth))
    for k,column in enumerate(columns): 
        r_x,r_y,r_width,r_higth = rects_local[k]
        time_series = (r_x,r_y, r_width,r_higth*0.9)
        ax_series =  fig.add_axes(time_series)

        # 时序图
        wrapped_title = textwrap.fill(column, width=subtitle_width)
        ax_series.set_title(wrapped_title, fontsize=fontsize_set)
        if column in label_columns:
            linewidth = 5
            markersize = 8
            color = "red" 
        else:
            linewidth = 2
            markersize = 5
            color = "#717d7e"
        
        ax_series.plot(data[x_column_index], data[column], color=color,  marker='o', markersize=5, linestyle='-', linewidth=linewidth)

        ax_series.tick_params(labelsize=fontsize_set-4, width=2, length=4, grid_linestyle=':')
        ax_series.grid()

        if ((k+1) / width_count - higth_count + 1) > 0:
            ax_series.set_xlabel('Stress Enhance', fontsize=fontsize_set-2, labelpad=15)

        # ax_series.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
        ax_series.ticklabel_format(axis='y', style='sci', scilimits=(0,4))

        wrapped_y_label = textwrap.fill(column, width=suby_label_width)
        ax_series.set_ylabel(wrapped_y_label, fontsize=fontsize_set-2, labelpad=15)
        
        
    plt.suptitle(f"{app} metirc", fontsize=fontsize_set+4)

    dir = f'{sub_dir}'
    if not os.path.exists(dir):
        os.makedirs(dir)
        
    fig.savefig(f'{dir}/{app}.png', dpi=200)
    plt.close(fig)

data = {
        'stress': [1, 2, 3, 4],
        'lat1': [10, 20, 15, 25],
        'lat2': [10, 20, 15, 25],
        'lat3': [10, 20, 15, 25],
        'lat4': [10, 20, 15, 25],
       }

df = pd.DataFrame(data)

compare(df,app="redis")
